In [13]:
import keras
import time
# import tensorflow as tf
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Flatten, Dropout, Activation, BatchNormalization, MaxPooling2D
from keras import optimizers
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.engine.topology import Input
from keras import backend as K
K.set_image_dim_ordering('tf')

In [11]:
# build the model from the scratch
input_img = Input((360, 360, 3))

#block 1
x = Conv2D(96, (11,11), strides=(4, 4), padding='valid')(input_img)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((3,3))(x)

#block 2
x = Conv2D(256, (5,5), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2,2))(x)

#block 3
x = Conv2D(256, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(256, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(256, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2,2))(x)

# fully-connected layers
x = Flatten()(x)
x = Dense(4096, activation = 'relu')(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation = 'relu')(x)
mlp = Dropout(0.5)(x)

model = Model(inputs = input_img, output = mlp)

plot_model(model, to_file='GazeNet_2.png')

/Users/ouyangjiahong/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:37: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("dr..., inputs=Tensor("in...)`


In [15]:
class GazeNet():
    def __init__(self,learning_rate):
        self.learning_rate = learning_rate
        self.model = self.create_mode
        self.batch_size = 32 # need to be discussed
        
    def create_mode(self,learning_rate):
      # build the model from the scratch
        input_img = Input((360, 360, 3))

        #block 1
        x = Conv2D(96, (11,11), strides=(4, 4), padding='valid')(input_img)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling2D((3,3))(x)

        #block 2
        x = Conv2D(256, (5,5), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling2D((2,2))(x)

        #block 3
        x = Conv2D(256, (3,3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv2D(256, (3,3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv2D(256, (3,3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling2D((2,2))(x)

        # fully-connected layers
        x = Flatten()(x)
        x = Dense(4096, activation = 'relu')(x)
        x = Dropout(0.5)(x)
        x = Dense(4096, activation = 'relu')(x)
        mlp = Dropout(0.5)(x)

        # CNN model end here
        
        LSTMLayer = keras.layers.LSTM(6,activation = 'tanh',recurrent_activation = 'hard_sigmoid',dropout = 0.5,recurrent_dropout = 0.5)(mlp)
        output = Dense(6,activation = 'softmax')(LSTMLayer)
        
        model = Model(inputs = input_img, outputs = output)
        plot_model(model, to_file='GazeNet_model_2.png')
        
        adam = optimizers.Adam(lr = self.learning_rate)
        model.compile(loss='categorical_crossentropy', optimizer='adam') # this line is not right, shouldn't be 'adam'

In [16]:
for layer in model.layers:
    print(layer.__class__.__name__)
    print(layer.output_shape)

InputLayer
(None, 360, 360, 3)
Conv2D
(None, 88, 88, 96)
BatchNormalization
(None, 88, 88, 96)
Activation
(None, 88, 88, 96)
MaxPooling2D
(None, 29, 29, 96)
Conv2D
(None, 29, 29, 256)
BatchNormalization
(None, 29, 29, 256)
Activation
(None, 29, 29, 256)
MaxPooling2D
(None, 14, 14, 256)
Conv2D
(None, 14, 14, 256)
BatchNormalization
(None, 14, 14, 256)
Activation
(None, 14, 14, 256)
Conv2D
(None, 14, 14, 256)
BatchNormalization
(None, 14, 14, 256)
Activation
(None, 14, 14, 256)
Conv2D
(None, 14, 14, 256)
BatchNormalization
(None, 14, 14, 256)
Activation
(None, 14, 14, 256)
MaxPooling2D
(None, 7, 7, 256)
Flatten
(None, 12544)
Dense
(None, 4096)
Dropout
(None, 4096)
Dense
(None, 4096)
Dropout
(None, 4096)
